# Operations Research Example

This file shows how to use SCGraph to generate a distance matrix for a set of cities and solve the TSP for that distance matrix using PULP (CBC Solver from COIN-OR).

In [ ]:
# pip install scgraph==2.14.0
# pip install pulp==3.3.0

In [1]:
from scgraph.geographs.us_freeway import us_freeway_geograph

# Define a list of cities with their latitude and longitude
cities = [
    {"latitude": 34.0522, "longitude": -118.2437},  # Los Angeles
    {"latitude": 40.7128, "longitude": -74.0060},   # New York City
    {"latitude": 41.8781, "longitude": -87.6298},   # Chicago
    {"latitude": 29.7604, "longitude": -95.3698},   # Houston
    {"latitude": 33.7490, "longitude": -84.3880},   # Atlanta
    {"latitude": 25.7617, "longitude": -80.1918},   # Miami
    {"latitude": 47.6062, "longitude": -122.3321},  # Seattle
    {"latitude": 44.9778, "longitude": -93.2650},   # Minneapolis
    {"latitude": 39.7392, "longitude": -104.9903},  # Denver
    {"latitude": 37.7749, "longitude": -122.4194},  # San Francisco
    {"latitude": 42.3601, "longitude": -71.0589},   # Boston
    {"latitude": 38.9072, "longitude": -77.0369},   # Washington DC
    {"latitude": 32.7765, "longitude": -79.9311},   # Charleston
    {"latitude": 29.9511, "longitude": -90.0715},   # New Orleans
]

# Define city names for output mapping
city_names = [
    "Los Angeles", "New York City", "Chicago", "Houston", "Atlanta",
    "Miami", "Seattle", "Minneapolis", "Denver", "San Francisco",
    "Boston", "Washington DC", "Charleston", "New Orleans"
]

# Compute the distance matrix using the US freeway geograph from SCGraph
distance_matrix = us_freeway_geograph.distance_matrix(cities, output_units='km')

In [2]:
import pulp
def solve_tsp(distance_matrix: list[list[float]]) -> tuple[list[int], float]:
    """
    Function: 

    - Solve the Traveling Salesman Problem (TSP) using Mixed Integer Linear Programming (MILP) in PuLP with the CBC solver from COIN-OR.

    Required Arguments:

    - distance_matrix: A 2D list or numpy array representing the distance between a pair of nodes.

    Returns:

    - tour: A list of node indices representing the order of the tour relative to the input distance matrix indices.
    - total_distance: The total distance of the optimal tour.
    """
    n = len(distance_matrix)
    cities = range(n)

    # Create the problem
    problem = pulp.LpProblem("TSP", pulp.LpMinimize)

    # Decision variables: x[i][j] = 1 if we travel from i to j
    x = pulp.LpVariable.dicts("x", (cities, cities), cat="Binary")
    # Store variable to track order and eliminate subtours
    order = pulp.LpVariable.dicts("u", cities, lowBound=0, upBound=n-1, cat="Continuous")

    # Objective: minimize total travel distance
    problem += pulp.lpSum(distance_matrix[i][j] * x[i][j] for i in cities for j in cities if i != j)

    # Constraints: leave each city exactly once
    for i in cities:
        problem += pulp.lpSum(x[i][j] for j in cities if i != j) == 1

    # Constraints: enter each city exactly once
    for j in cities:
        problem += pulp.lpSum(x[i][j] for i in cities if i != j) == 1

    # Subtour elimination constraints (MTZ formulation)
    for i in cities:
        for j in cities:
            if i != j and i != 0 and j != 0:
                problem += order[i] - order[j] + (n - 1) * x[i][j] <= n - 2

    # Solve
    solver = pulp.PULP_CBC_CMD(msg=0)
    problem.solve(solver)

    # Extract solution
    tour = [0]
    current_city = 0
    while len(tour) < n:
        for j in cities:
            if j != current_city and pulp.value(x[current_city][j]) == 1:
                tour.append(j)
                current_city = j
                break

    # Close the tour
    tour.append(0)

    total_distance = pulp.value(problem.objective)
    return tour, total_distance

In [3]:
# Solve the TSP for the given distance matrix
tour, total_distance = solve_tsp(distance_matrix)

# Print the optimal tour and its total distance by mapping indices to city names
print(f"Optimal tour ({round(total_distance)} km):\n->", "\n-> ".join(city_names[i] for i in tour))

Optimal tour (15369 km):
-> Los Angeles
-> Houston
-> New Orleans
-> Atlanta
-> Miami
-> Charleston
-> Washington DC
-> New York City
-> Boston
-> Chicago
-> Minneapolis
-> Denver
-> Seattle
-> San Francisco
-> Los Angeles
